In [11]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm

In [ ]:
# https://gist.github.com/twiecki/14694749b5f85e89b246

In [58]:
alpha, sigma = 1, 1
beta = [1, 2.5]
N = 10000
X1 = np.random.randn(N)
X2 = np.random.randn(N) * 0.2
Y = alpha + beta[0] * X1 + beta[1] * X2 + np.random.randn(N) * sigma

with pm.Model() as model:
    alpha = pm.Normal('alpha', mu = 0, sd = 10)
    beta = pm.Normal('beta', mu = 0, sd = 10, shape = 2)
    sigma = pm.HalfNormal('sigma', sd = 1)
    mu = alpha + beta[0] * X1 + beta[1] * X2
    Y_obs = pm.Normal('Y_obs', mu = mu, sd = sigma, observed = Y)
    
with model:
    start = pm.find_MAP()
    trace = pm.sample(2000, start = start)
    
trace['alpha'][:5]

Applied log-transform to sigma and added transformed sigma_log_ to model.
Assigned NUTS to alpha
Assigned NUTS to beta
Assigned NUTS to sigma_log_
100%|██████████| 2000/2000 [00:04<00:00, 489.17it/s]


array([ 0.99959347,  0.99959347,  0.99143305,  0.99804502,  1.00746962])

In [60]:
data = np.ma.masked_values([
    4, 5, 4, 0, 1, 4, 3, 4, 0, 6, 3, 3, 4, 0, 2, 6,
    3, 3, 5, 4, 5, 3, 1, 4, 4, 1, 5, 5, 3, 4, 2, 5,
    2, 2, 3, 4, 2, 1, 3, -999, 2, 1, 1, 1, 1, 3, 0, 0,
    1, 0, 1, 1, 0, 0, 3, 1, 0, 3, 2, 2, 0, 1, 1, 1,
    0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 0, 0, 1, 1, 0, 2,
    3, 3, 1, -999, 2, 1, 1, 1, 1, 2, 4, 2, 0, 0, 1, 4,
    0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1
], value = -999)

year = np.arange(1851, 1962)

with Model() as model:
    switchpoint = pm.DiscreteUniform('switchpoint', lower = year.min(), upper = year.max(), testval = 1900)
    early_rate = pm.Exponential('early_rate', 1)
    late_rate = pm.Exponential('late_rate', 1)
    rate = pm.math.switch(switchpoint >= year, early_rate, late_rate)
    disasters = pm.Poisson('disasters', rate, observed = data)
    
with model:
    step1 = pm.NUTS([early_rate, late_rate])
    step2 = pm.Metropolis([switchpoint, disasters.missing_values[0]])
    trace = pm.sample(10000, step = [step1, step2])

Applied log-transform to early_rate and added transformed early_rate_log_ to model.
Applied log-transform to late_rate and added transformed late_rate_log_ to model.
100%|██████████| 10000/10000 [00:10<00:00, 985.68it/s]


In [61]:
# from
# https://github.com/CamDavidsonPilon/Probabilistic-Programming-and-Bayesian-Methods-for-Hackers
# /blob/master/Chapter1_Introduction/Ch1_Introduction_PyMC3.ipynb

url = 'https://raw.githubusercontent.com/CamDavidsonPilon/'
url += 'Probabilistic-Programming-and-Bayesian-Methods-for-Hackers/master/'
url += 'Chapter1_Introduction/data/txtdata.csv'
data = pd.read_table(url, header = None).iloc[:, 0].values
N = len(data)
alpha = 1.0 / data.mean()

with pm.Model() as model:
    lambda_1 = pm.Exponential('lambda_1', alpha)
    lambda_2 = pm.Exponential('lambda_2', alpha)
    tau = pm.DiscreteUniform('tau', lower = 0, upper = N)
    
with model:
    idx = np.arange(N)
    lambda_ = pm.math.switch(tau >= idx, lambda_1, lambda_2)    

Applied log-transform to lambda_1 and added transformed lambda_1_log_ to model.
Applied log-transform to lambda_2 and added transformed lambda_2_log_ to model.
